In [19]:
%reload_ext autoreload
%autoreload 2

import os

import sys
sys.path.append(os.environ['GORDON_REPO_DIR'] + '/pipeline_scripts')

from utilities2014 import *

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=1)
dm._load_image()

# Over-segment the image into superpixels using SLIC (http://ivrg.epfl.ch/research/superpixels)

from skimage.segmentation import slic, mark_boundaries
from skimage.util import img_as_ubyte, pad
from joblib import Parallel, delayed

masked_image = dm.image.copy()
masked_image[~dm.mask] = 0

textonmap2 = dm.load_pipeline_result('texMap', 'npy')
textonmap_viz = dm.load_pipeline_result('texMap', 'png')
n_texton = len(np.unique(textonmap2)) - 1


import collections
from skimage.filters import gaussian_filter
from _regular_grid import regular_grid

import _slic_texture


In [98]:
# ymin, ymax, xmin, xmax = (2140, 2511, 900, 1322)
# ymin, ymax, xmin, xmax = (1064, 1700, 4805, 5300)
ymin, ymax, xmin, xmax = (1340, 1800, 2640, 3080)
grids = (slice(ymin, ymax, 1), slice(xmin, xmax, 1))

image = img_as_float(masked_image[grids]).copy()
textonmap = textonmap2[grids].copy()

In [99]:
spacing = None
# sigma = 1.
n_segments = 100
# compactness = 1.
max_iter = 10

if spacing is None:
    spacing = np.ones(3)
elif isinstance(spacing, (list, tuple)):
    spacing = np.array(spacing, dtype=np.double)

# image = gaussian_filter(image, sigma)

height, width = image.shape[:2]

# initialize cluster centroids for desired number of segments
grid_y, grid_x = np.mgrid[:height, :width]
slices = regular_grid(image.shape[:2], n_segments)
step_y, step_x = [int(s.step) for s in slices]
segments_y = grid_y[slices]
segments_x = grid_x[slices]

image_texture = np.zeros((image.shape + (n_texton,)))
for y in range(image.shape[0]):
    for x in range(image.shape[1]):
        if textonmap[y,x] != -1:
            image_texture[y, x, textonmap[y,x]] = 1

segments_texture = image_texture[slices]
            
segments = np.concatenate([segments_y[..., np.newaxis],
                           segments_x[..., np.newaxis],
                           segments_texture],
                          axis=-1).reshape(-1, 2+n_texton)
segments = np.ascontiguousarray(segments)
    
# we do the scaling of ratio in the same way as in the SLIC paper
# so the values have the same meaning
step = float(max((step_y, step_x)))
# ratio = 1.0 / compactness

# image = np.ascontiguousarray(image * ratio)
# if enforce_connectivity:
#     segment_size = depth * height * width / n_segments
#     min_size = int(min_size_factor * segment_size)
#     max_size = int(max_size_factor * segment_size)
#     labels = _enforce_label_connectivity_cython(labels,
#                                                 n_segments,
#                                                 min_size,
#                                                 max_size)

In [100]:
labels = _slic_texture._slic_cython(image_texture, segments, step, max_iter, spacing, False)

min_size_factor=0.5
max_size_factor=3

segment_size = height * width / n_segments
min_size = int(min_size_factor * segment_size)
max_size = int(max_size_factor * segment_size)
labels = _slic_texture._enforce_label_connectivity_cython(labels,
                                            n_segments,
                                            min_size,
                                            max_size)

labels2 = slic(gray2rgb(img_as_float(masked_image[grids])), 
                n_segments=n_segments, 
                max_iter=10, 
#                     compactness=float(dm.segm_params['slic_compactness']), 
                compactness=10., 
                sigma=float(dm.segm_params['slic_sigma']), 
                enforce_connectivity=True)



labels3 = slic(gray2rgb(img_as_float(masked_image[grids])), 
                n_segments=n_segments, 
                max_iter=10, 
#                     compactness=float(dm.segm_params['slic_compactness']), 
                compactness=30., 
                sigma=float(dm.segm_params['slic_sigma']), 
                enforce_connectivity=True)

labels4 = slic(gray2rgb(img_as_float(masked_image[grids])), 
                n_segments=n_segments, 
                max_iter=10, 
                compactness=float(dm.segm_params['slic_compactness']), 
                sigma=float(dm.segm_params['slic_sigma']), 
                enforce_connectivity=True)


a = gray2rgb(img_as_float(masked_image[grids]))

b = mark_boundaries(a, labels)

bb = mark_boundaries(img_as_float(textonmap_viz[grids]), labels)

c = mark_boundaries(a, labels2)
d = mark_boundaries(a, labels3)
e = mark_boundaries(a, labels4)

G_nonmaxsup = dm.load_pipeline_result('Gnonmaxsup', 'npy')[grids]
Gmax = dm.load_pipeline_result('Gmax', 'npy')[grids]
alpha = 1.5
Gmax = 1 - 1/(1+alpha*Gmax)

G_nonmaxsup_thresh_viz = b.copy()
for y,x in zip(*np.where(G_nonmaxsup)):
    if Gmax[y,x] > .6:
        r = int(Gmax[y,x]*5)
        G_nonmaxsup_thresh_viz[y-r:y+r, x-r:x+r] = [Gmax[y,x], 0, 0]
        
f = G_nonmaxsup_thresh_viz

# b = colors[labels]
display(np.concatenate([a, bb, b, f, c, d, e], axis=1))

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [31]:
import matplotlib.pyplot as plt

In [10]:
# colors = np.loadtxt('../visualization/100colors.txt')